# **Learn OpenUSD: Experimenting With VariantSets**

Enjoy this bonus module, *Learn OpenUSD: Experimenting With VariantSets* to wrap up our **Learn OpenUSD: Foundations** series. 

>**NOTE**: Before starting make sure to run the cell below. This will install the relevant OpenUSD libraries that will be used through this notebook.

In [1]:
from utils.visualization import DisplayUSD
from utils.helperfunctions import create_new_stage

Before we begin, let's create a USD stage using the [`stage`](https://openusd.org/release/glossary.html#usdglossary-stage) class from the `Usd` module.


**Run the cell below to create a new USD Stage.** This will setup a stage that contains a variety of prims of different schemas. We'll use this content to demonstrate how to build `VariantSets` in the rest of this module.

In [4]:
from pxr import Usd, UsdGeom

stage: Usd.Stage = create_new_stage("assets/variant_prims.usda")

world: UsdGeom.Xform = UsdGeom.Xform.Define(stage, "/World")
stage.SetDefaultPrim(world.GetPrim())

box: UsdGeom.Xform = UsdGeom.Xform.Define(stage, world.GetPath().AppendPath("Box"))
geo_scope: UsdGeom.Scope = UsdGeom.Scope.Define(stage, box.GetPath().AppendPath("Geometry"))

stage.Save()
DisplayUSD("assets/variant_prims.usda", show_usd_code=True)

---

## **Activity 1**: Verifying VariantSets Exist

A [`variantSet`](https://openusd.org/release/glossary.html#usdglossary-variantset) is like a group of options that you can apply to a single item (called a "prim"). Imagine it as a switch that lets a content creator bundle different choices together. This way, someone using the content later can easily switch between these options or add new ones without changing the original item.

To check if a variantSet exists, use the [`HasVariantSets()`](https://openusd.org/release/api/class_usd_prim.html#a87443b32a72f95ca96d960b4e96cbf02) method.

**Run the cell below to check if the variantSet exists in the `variant_prims.usda` we created in the previous cell.**

In [5]:
from pxr import Usd

# Open the USD stage from the specified file:
stage = Usd.Stage.Open("assets/variant_prims.usda")

# Get the prim at the specified path:
geo_scope = stage.GetPrimAtPath("/World/Box/Geometry")

# Print whether the prim has variant sets:
print(geo_scope.HasVariantSets())

False


---

## **Activity 2**: Adding a VariantSet and Variants

[`Variants`](https://openusd.org/release/glossary.html#usdglossary-variant) can hold different settings or details, like properties or metadata, and even entire sections of a scene. A variant is just one specific version within a group called a variantSet. Each variantSet can have one or none of these versions selected in a scene.

To create a variantSet, it needs to have at least one variant in the variantSet. 

To get all varaintSets, we would use [`GetVariantSets()`](https://openusd.org/release/api/class_usd_prim.html#a607da249e11bc4f5f3b4bf0db99861ab). 

To add a variantSet, we'd use [`AddVariantSet()`](https://openusd.org/release/api/class_usd_variant_sets.html#afa72971becc14b53366f2913307b8164). 

For adding variants to the variantSet, we would use [`AddVariant()`](https://openusd.org/release/api/class_usd_variant_set.html#a13cac327430d050108a8c22cada16b45).

A small example on how to use the methods can be found below:

```python
from pxr import Usd

# Open the USD stage from the specified file:
stage = Usd.Stage.Open("assets/variant_prims.usda")

# Get the prim at the specified path:
geo_scope = stage.GetPrimAtPath("/World/Box/Geometry")

# Get the "shapes" variant set for the geometry scope:
geo_variant_sets = geo_scope.GetVariantSets()
shapes_variant_set = geo_variant_sets.AddVariantSet("shapes")

# Add a variant named "Cube" to the "shapes" variant set:
geo_variant_set.AddVariant("Cube")

# Print whether the geometry scope now has variant sets:
print(geo_scope.HasVariantSets())

# Save the changes to the stage:
stage.Save()
```

Let's add some variants to the `shapes` variant set.

**Add the following code to the cell below, then run the cell:**

```python
geo_variant_sets = geo_scope.GetVariantSets()
shapes_variant_set = geo_variant_sets.AddVariantSet("shapes")
for shape in shapes:
    shapes_variant_set.AddVariant(shape)
```

In [6]:
from pxr import Usd

# Open the USD stage from the specified file:
stage = Usd.Stage.Open("assets/variant_prims.usda")

# List of shapes to add as variants:
shapes = ["Cube", "Sphere", "Cylinder", "Cone"]

# Get the prim at the specified path "/World/Box/Geometry":
geo_scope = stage.GetPrimAtPath("/World/Box/Geometry")

geo_variant_sets = geo_scope.GetVariantSets()
shapes_variant_set = geo_variant_sets.AddVariantSet("shapes")
for shape in shapes:
    shapes_variant_set.AddVariant(shape)


# Save the changes to the stage:
stage.Save()
# Print the unflattened Root Layer:
DisplayUSD("assets/variant_prims.usda", show_usd_code=True)

---

## **Activity 3**: Modifying Variants

To edit a variant, switch to it by using the [`SetVariantSelection()`](https://openusd.org/release/api/class_usd_variant_set.html#a51fe1abe65de6440b81799393b1a424f) method. Then, we can edit the variant and make our changes, such as modifying properties or adding new details. 

In our case, we will be modifying what shape will be defined under the geometry scope.

**Add the following code to the cell below, then run the cell:**

```python
# Loop over each shape in the list of shapes
for shape in shapes:
    # Print the shape being added
    print("adding: " + shape)
    
    # Add a variant named after the shape to the "shapes" variant set
    shapes_variant_set.AddVariant(shape)
    
    # Select the current variant for editing
    shapes_variant_set.SetVariantSelection(shape)
    
    # Enter the variant edit context to make changes specific to the current variant
    with shapes_variant_set.GetVariantEditContext():
        # Define a new prim for the current shape under the geometry scope
        shape_prim = stage.DefinePrim(geo_scope.GetPath().AppendPath(shape))
        
        # Set the type of the new prim to the current shape
        shape_prim.SetTypeName(shape)
```

In [7]:
from pxr import Usd

# Open the USD stage from the specified file:
stage = Usd.Stage.Open("assets/variant_prims.usda")

# List of shapes to add as variants:
shapes = ["Cube", "Sphere", "Cylinder", "Cone"]

# Get the prim at the specified path "/World/Box/Geometry":
geo_scope = stage.GetPrimAtPath("/World/Box/Geometry")

# Get the "shapes" variant set for the geometry scope:
geo_variant_sets = geo_scope.GetVariantSets()
shapes_variant_set = geo_variant_sets.AddVariantSet("shapes")
    
# Loop over each shape in the list of shapes
for shape in shapes:
    # Print the shape being added
    print("adding: " + shape)
    
    # Add a variant named after the shape to the "shapes" variant set
    shapes_variant_set.AddVariant(shape)
    
    # Select the current variant for editing
    shapes_variant_set.SetVariantSelection(shape)
    
    # Enter the variant edit context to make changes specific to the current variant
    with shapes_variant_set.GetVariantEditContext():
        # Define a new prim for the current shape under the geometry scope
        shape_prim = stage.DefinePrim(geo_scope.GetPath().AppendPath(shape))
        
        # Set the type of the new prim to the current shape
        shape_prim.SetTypeName(shape)


# Here we define what Variant from the VariantSet is selected. Change this between "Cube", "Sphere", "Cylinder", and "Cone"
# to see the different geometries:
shapes_variant_set.SetVariantSelection("Cube")
# shapes_variant_set.SetVariantSelection("Sphere")
# shapes_variant_set.SetVariantSelection("Cylinder")
# shapes_variant_set.SetVariantSelection("Cone")

# Save the changes to the stage
stage.Save()
DisplayUSD("assets/variant_prims.usda", show_usd_code=True)

adding: Cube
adding: Sphere
adding: Cylinder
adding: Cone


In OpenUSD, `VariantSets` and `Variants` enable the creation of flexible, dynamic scenes by allowing multiple versions of models or components to coexist within a single scene graph. This facilitates efficient asset management, easy customization, and the ability to switch between different configurations seamlessly.